# CLX Asset Classification (Supervised)

## Authors
- Eli Fajardo (NVIDIA)
- Görkem Batmaz (NVIDIA)
- Bhargav Suryadevara (NVIDIA)


## Table of Contents 
* Introduction
* Dataset
* Reading in the datasets
* Training and inference
* References

# Introduction

In this notebook, we will show how to predict the function of a server with Windows Event Logs using cudf, cuml and pytorch. The machines are labeled as DC, SQL, WEB, DHCP, MAIL and SAP. The dependent variable will be the type of the machine. The features are selected from Windows Event Logs which is in a tabular format. This is a first step to learn the behaviours of certain types of machines in data-centres by classifying them probabilistically. It could help to detect unusual behaviour in a data-centre. For example, some compromised computers might be acting as web/database servers but with their original tag. 

This work could be expanded by using different log types or different events from the machines as features to improve accuracy. Various labels can be selected to cover different types of machines or data-centres.

## Library imports

In [1]:
from clx.analytics.asset_classification import AssetClassification
import cudf
from cuml.preprocessing import train_test_split
from cuml.preprocessing import LabelEncoder
import torch
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd
from os import path
import s3fs

## Initialize variables

10000 is chosen as the batch size to optimise the performance for this dataset. It can be changed depending on the data loading mechanism or the setup used. 

EPOCH should also be adjusted depending on convergence for a specific dataset. 

label_col indicates the total number of features used plus the dependent variable. Feature names are listed below.

In [2]:
batch_size = 3000
label_col = '19'
epochs = 15

In [3]:
ac = AssetClassification()

## Read the dataset into a GPU dataframe with `cudf.read_csv()` 

The original data had many other fields. Many of them were either static or mostly blank. After filtering those, there were 18 meaningful columns left. In this notebook we use a fake continuous feature to show the inclusion of continuous features too. When you are using raw data the cell below need to be uncommented

In [4]:
# win_events_gdf = cudf.read_csv("raw_features_and_labels.csv")

```
win_events_gdf.dtypes

eventcode                                                       int64
keywords                                                       object
privileges                                                     object
message                                                        object
sourcename                                                     object
taskcategory                                                   object
account_for_which_logon_failed_account_domain                  object
detailed_authentication_information_authentication_package     object
detailed_authentication_information_key_length                float64
detailed_authentication_information_logon_process              object
detailed_authentication_information_package_name_ntlm_only     object
logon_type                                                    float64
network_information_workstation_name                           object
new_logon_security_id                                          object
impersonation_level                                            object
network_information_protocol                                  float64
network_information_direction                                  object
filter_information_layer_name                                  object
cont1                                                           int64
label                                                          object
dtype: object
```

### Define categorical and continuous feature columns.

In [5]:
cat_cols = [
    "eventcode",
    "keywords",
    "privileges",
    "message",
    "sourcename",
    "taskcategory",
    "account_for_which_logon_failed_account_domain",
    "detailed_authentication_information_authentication_package",
    "detailed_authentication_information_key_length",
    "detailed_authentication_information_logon_process",
    "detailed_authentication_information_package_name_ntlm_only",
    "logon_type",
    "network_information_workstation_name",
    "new_logon_security_id",
    "impersonation_level",
    "network_information_protocol",
    "network_information_direction",
    "filter_information_layer_name",
    "label"
]

In [6]:
cont_cols = [
    "cont1"
]

The following are functions used to preprocess categorical and continuous feature columns. This can very depending on what best fits your application and data.

In [7]:
def categorize_columns(cat_gdf):
    for col in cat_gdf.columns:
        cat_gdf[col] = cat_gdf[col].astype('str')
        cat_gdf[col] = cat_gdf[col].fillna("NA")
        cat_gdf[col] = LabelEncoder().fit_transform(cat_gdf[col])
        cat_gdf[col] = cat_gdf[col].astype('int16')
        
    return cat_gdf

In [8]:
def normalize_conts(cont_gdf):
    means, stds = (cont_gdf.mean(0), cont_gdf.std(ddof=0))
    cont_gdf = (cont_gdf - means) / stds
    
    return cont_gdf

Preprocessing steps below are not executed in this notebook, because we release already preprocessed data.

In [9]:
#win_events_gdf[cat_cols] = categorize_columns(win_events_gdf[cat_cols])

In [10]:
#win_events_gdf[cont_cols] = normalize_conts(win_events_gdf[cont_cols])

Read Windows Event data already preprocessed by above steps

In [11]:
S3_BASE_PATH = "rapidsai-data/cyber/clx"
WINEVT_PREPROC_CSV = "win_events_features_preproc.csv"

# Download Zeek conn log
if not path.exists(WINEVT_PREPROC_CSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + WINEVT_PREPROC_CSV, WINEVT_PREPROC_CSV)

win_events_gdf = cudf.read_csv("win_events_features_preproc.csv")

In [12]:
win_events_gdf.head()

,eventcode,keywords,privileges,message,sourcename,taskcategory,account_for_which_logon_failed_account_domain,detailed_authentication_information_authentication_package,detailed_authentication_information_key_length,detailed_authentication_information_logon_process,detailed_authentication_information_package_name_ntlm_only,logon_type,network_information_workstation_name,new_logon_security_id,impersonation_level,network_information_protocol,network_information_direction,filter_information_layer_name,cont1,label
0,0,1,0,15,0,4,22,0,0,5,0,1,932,38,3,6,1,1,-1.732030,1
1,14,1,0,7,0,5,22,3,2,6,1,6,932,25,3,6,1,1,-1.731988,0
2,14,1,0,7,0,5,22,3,2,6,1,6,932,25,3,6,1,1,-1.731945,0
3,14,1,0,7,0,5,22,3,2,6,1,6,932,25,3,6,1,1,-1.731903,0
4,14,1,0,7,0,5,22,3,2,6,1,6,932,25,3,6,1,1,-1.731861,0


### Split the dataset into training and test sets using cuML `train_test_split` function
Column 19 contains the ground truth about each machine's function that the logs come from. i.e. DC, SQL, WEB, DHCP, MAIL and SAP. Hence it will be used as a label.

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(win_events_gdf, "label", train_size=0.9)
X_train["label"] = Y_train

In [14]:
X_train.head()

,eventcode,keywords,privileges,message,sourcename,taskcategory,account_for_which_logon_failed_account_domain,detailed_authentication_information_authentication_package,detailed_authentication_information_key_length,detailed_authentication_information_logon_process,detailed_authentication_information_package_name_ntlm_only,logon_type,network_information_workstation_name,new_logon_security_id,impersonation_level,network_information_protocol,network_information_direction,filter_information_layer_name,cont1,label
23478,0,1,0,15,0,4,22,0,0,5,0,1,0,2194,2,6,1,1,-0.741262,0
46780,0,1,0,15,0,4,22,0,0,5,0,1,0,7159,2,6,1,1,0.242079,1
17616,0,1,0,15,0,4,22,0,0,5,0,1,0,3983,2,6,1,1,-0.988637,0
53286,0,1,0,15,0,4,22,0,0,5,0,1,932,2109,2,6,1,1,0.516632,2
33927,0,1,0,15,0,4,22,0,0,5,0,1,0,8682,0,6,1,1,-0.300315,0


In [15]:
Y_train.unique()

0    0
1    1
2    2
3    3
4    4
5    5
Name: label, dtype: int64

### Print Labels
Making sure the test set contains all labels

In [16]:
Y_test.unique()

0    0
1    1
2    2
3    3
4    4
5    5
Name: label, dtype: int64

## Training 

Asset Classification training uses the fastai tabular model. More details can be found at https://github.com/fastai/fastai/blob/master/fastai/tabular/models.py#L6

Feature columns will be embedded so that they can be used as categorical values. The limit can be changed depending on the accuracy of the dataset.

Adam is the optimizer used in the training process; it is popular because it produces good results in various tasks. In its paper, computing the first and the second moment estimates and updating the parameters are summarized as follows

$$\alpha_{t}=\alpha \cdot \sqrt{1-\beta_{2}^{t}} /\left(1-\beta_{1}^{t}\right)$$

More detailson Adam can be found at https://arxiv.org/pdf/1412.6980.pdf

We have found that the way we partition the dataframes with a 10000 batch size gives us the optimum data loading capability. The **batch_size** argument can be adjusted for different sizes of datasets.

In [17]:
cat_cols.remove("label")
ac.train_model(X_train, cat_cols, cont_cols, "label", batch_size, epochs, lr=0.01, wd=0.0)

training loss:  1.0111992062101818
valid loss 0.583 and accuracy 0.814
training loss:  0.4620857691055216
valid loss 0.391 and accuracy 0.876
training loss:  0.33254117653094556
valid loss 0.312 and accuracy 0.902
training loss:  0.28072822153262583
valid loss 0.279 and accuracy 0.910
training loss:  0.2554837790583415
valid loss 0.263 and accuracy 0.914
training loss:  0.2408174945388092
valid loss 0.250 and accuracy 0.915
training loss:  0.23049960875422962
valid loss 0.244 and accuracy 0.916
training loss:  0.2221764272199862
valid loss 0.238 and accuracy 0.918
training loss:  0.2154606228360371
valid loss 0.234 and accuracy 0.919
training loss:  0.210641215422796
valid loss 0.233 and accuracy 0.921
training loss:  0.2069480326228095
valid loss 0.234 and accuracy 0.922
training loss:  0.20380194447335698
valid loss 0.238 and accuracy 0.923
training loss:  0.20021527777256393
valid loss 0.236 and accuracy 0.923
training loss:  0.19645206474967966
valid loss 0.230 and accuracy 0.923
t

## Evaluation

In [18]:
pred_results = ac.predict(X_test, cat_cols, cont_cols).to_arrow().to_pylist()
true_results = Y_test.to_arrow().to_pylist()

In [19]:
f1_score_ = f1_score(pred_results, true_results, average='micro')
print('micro F1 score: %s'%(f1_score_))

micro F1 score: 0.9260567669630893


In [20]:
torch.cuda.empty_cache()

In [21]:
labels = ["DC","DHCP","MAIL","SAP","SQL","WEB"]
a = confusion_matrix(true_results, pred_results)

In [22]:
pd.DataFrame(a, index=labels, columns=labels)

,DC,DHCP,MAIL,SAP,SQL,WEB
DC,3525,35,10,7,84,11
DHCP,93,660,1,2,13,0
MAIL,11,0,2555,6,7,0
SAP,10,0,1,151,9,0
SQL,194,1,9,19,643,24
WEB,21,0,0,1,38,68


The confusion matrix shows that some machines' function can be predicted really well, whereas some of them need more tuning or more features. This work can be improved and expanded to cover individual data-centres to create a realistic map of the network using ML by not just relying on the naming conventions. It could also help to detect more prominent scale anomalies like multiple machines, not acting per their tag.

## References:
* https://github.com/fastai/fastai/blob/master/fastai/tabular/models.py#L6
* https://jovian.ml/aakashns/04-feedforward-nn
* https://www.kaggle.com/dienhoa/reverse-tabular-module-of-fast-ai-v1
* https://github.com/fastai/fastai/blob/master/fastai/layers.py#L44